In [ ]:
# 1. Kết nối Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Cài đặt thư viện
!pip install -q accelerate transformers wandb mediapipe

# 3. (Giả sử) Tải code của chị từ GitHub
# !git clone https://github.com/ten-cua-chi/ten-repo.git
# %cd /content/ten-repo/

# 4. Định nghĩa đường dẫn trỏ vào Google Drive
# (Thay 'SLP_Project' bằng tên thư mục của chị)
%env DATA_ROOT /content/drive/MyDrive/SLP_Project/data/final
%env OUTPUT_ROOT /content/drive/MyDrive/SLP_Project/checkpoints
%env WANDB_PROJECT slp-hierarchical-flow-phoenix

# 5. Tạo các thư mục checkpoints
!mkdir -p $OUTPUT_ROOT/manual_flow
!mkdir -p $OUTPUT_ROOT/nmm_flow
!mkdir -p $OUTPUT_ROOT/sync

In [ ]:
# GIAI ĐOẠN 1: TRAIN MANUAL FLOW (Tay/Thân)
!accelerate launch training/train_manual_flow.py \
    --data_root $DATA_ROOT \
    --output_dir $OUTPUT_ROOT/manual_flow \
    --batch_size 4 \
    --num_epochs 50 \
    --learning_rate 1e-4 \
    --wandb_project $WANDB_PROJECT \
    --wandb_run_name "manual_flow_hierarchical"
    
# (Nếu bị "sập" giữa chừng, chị thêm dòng này vào lệnh trên và chạy lại:)
# --resume_from $OUTPUT_ROOT/manual_flow/checkpoint_epoch_XX.pt

In [ ]:
# GIAI ĐOẠN 2: TRAIN NMM FLOW (Mặt/Cảm xúc)
!accelerate launch training/train_nmm_flow.py \
    --data_root $DATA_ROOT \
    --output_dir $OUTPUT_ROOT/nmm_flow \
    --batch_size 8 \
    --num_epochs 40 \
    --learning_rate 1e-4 \
    --share_text_encoder $OUTPUT_ROOT/manual_flow/best_model.pt \
    --freeze_text_encoder \
    --wandb_project $WANDB_PROJECT \
    --wandb_run_name "nmm_flow_all"

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🚀 (Cell 0) Cài đặt và Kết nối Google Drive\n",
    "Chạy cell này 1 lần duy nhất mỗi khi khởi động Colab."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Kết nối với Google Drive\n",
    "from google.colab import drive\n",
    "drive.mount('/content/drive')\n",
    "\n",
    "# 2. Cài đặt các thư viện cần thiết\n",
    "!pip install -q accelerate transformers wandb mediapipe pandas\n",
    "\n",
    "# 3. Clone code của chị từ GitHub (hoặc upload)\n",
    "# (Bỏ comment và thay bằng link Git của chị)\n",
    "# !git clone https://github.com/ten-cua-chi/ten-repo.git\n",
    "# %cd /content/ten-repo/\n",
    "\n",
    "# 4. (RẤT QUAN TRỌNG) Định nghĩa đường dẫn trên Google Drive\n",
    "# (Thay 'SLP_Project' bằng tên thư mục của chị)\n",
    "import os\n",
    "os.environ['DATA_ROOT'] = '/content/drive/MyDrive/SLP_Project/data/final'\n",
    "os.environ['OUTPUT_ROOT'] = '/content/drive/MyDrive/SLP_Project/checkpoints'\n",
    "os.environ['WANDB_PROJECT'] = 'slp-hierarchical-flow-phoenix'\n",
    "\n",
    "# 5. Tạo các thư mục checkpoints (nếu chưa có)\n",
    "!mkdir -p $OUTPUT_ROOT/manual_flow\n",
    "!mkdir -p $OUTPUT_ROOT/nmm_flow\n",
    "!mkdir -p $OUTPUT_ROOT/sync\n",
    "\n",
    "print(\"✅ Setup hoàn tất! Sẵn sàng train.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---",
    "## 🏃‍♂️ (Cell 1) GIAI ĐOẠN 1: Train Manual Flow (Tay/Thân)\n",
    "Chạy cell này (có thể mất 1-2 ngày). Nếu bị ngắt kết nối:\n",
    "1. Chạy lại Cell 0.\n",
    "2. Thêm cờ `--resume_from $OUTPUT_ROOT/manual_flow/checkpoint_epoch_XX.pt` vào lệnh dưới.\n",
    "3. Chạy lại Cell 1."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!accelerate launch training/train_manual_flow.py \\\n",
    "    --data_root $DATA_ROOT \\\n",
    "    --output_dir $OUTPUT_ROOT/manual_flow \\\n",
    "    --batch_size 4 \\\n",
    "    --num_epochs 50 \\\n",
    "    --learning_rate 1e-4 \\\n",
    "    --wandb_project $WANDB_PROJECT \\\n",
    "    --wandb_run_name \"manual_flow_hierarchical\" \n",
    "\n",
    "# (Nếu bị sập, thêm dòng này vào lệnh trên và chạy lại:)\n",
    "# --resume_from $OUTPUT_ROOT/manual_flow/checkpoint_epoch_10.pt"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---",
    "## 🎭 (Cell 2) GIAI ĐOẠN 2: Train NMM Flow (Mặt/Cảm xúc)\n",
    "**CHỈ CHẠY** sau khi Giai đoạn 1 hoàn thành và có file `best_model.pt`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!accelerate launch training/train_nmm_flow.py \\\n",
    "    --data_root $DATA_ROOT \\\n",
    "    --output_dir $OUTPUT_ROOT/nmm_flow \\\n",
    "    --batch_size 8 \\\n",
    "    --num_epochs 40 \\\n",
    "    --learning_rate 1e-4 \\\n",
    "    --share_text_encoder $OUTPUT_ROOT/manual_flow/best_model.pt \\\n",
    "    --freeze_text_encoder \\\n",
    "    --wandb_project $WANDB_PROJECT \\\n",
    "    --wandb_run_name \"nmm_flow_all\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---",
    "## 🔗 (Cell 3) GIAI ĐOẠN 3: Train Joint Synchronization\n",
    "**CHỈ CHẠY** sau khi Giai đoạn 1 VÀ 2 đều hoàn thành."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!accelerate launch training/train_joint_sync.py \\\n",
    "    --data_root $DATA_ROOT \\\n",
    "    --manual_checkpoint $OUTPUT_ROOT/manual_flow/best_model.pt \\\n",
    "    --nmm_checkpoint $OUTPUT_ROOT/nmm_flow/best_model.pt \\\n",
    "    --output_dir $OUTPUT_ROOT/sync \\\n",
    "    --batch_size 4 \\\n",
    "    --num_epochs 30 \\\n",
    "    --learning_rate 5e-5 \\\n",
    "    --wandb_project $WANDB_PROJECT \\\n",
    "    --wandb_run_name \"joint_sync\""
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


In [ ]:
# GIAI ĐOẠN 3: TRAIN "KEO DÁN" (Sync)
!accelerate launch training/train_joint_sync.py \
    --data_root $DATA_ROOT \
    --manual_checkpoint $OUTPUT_ROOT/manual_flow/best_model.pt \
    --nmm_checkpoint $OUTPUT_ROOT/nmm_flow/best_model.pt \
    --output_dir $OUTPUT_ROOT/sync \
    --batch_size 4 \
    --num_epochs 30 \
    --learning_rate 5e-5 \
    --wandb_project $WANDB_PROJECT \
    --wandb_run_name "joint_sync"